In [50]:
import sys
import os

chemin_actuel = os.getcwd()
chemin_parent = os.path.dirname(chemin_actuel)
sys.path.append(chemin_parent)

from class_sportif import *
from algo_matching_sportifs import *

In [51]:
def func_test_calculer_distance():
    
    # Définition d'une tolérance pour les comparaisons en virgule flottante
    TOLERANCE = 0.01 # Tolérance de 10 mètres pour la distance en km

    print("--- 🧪 Début des Tests Manuels de Distance Haversine ---")

    # ==========================================================
    ## 🎯 SCÉNARIO 1 : Distance nulle (Même point)
    # ==========================================================
    print("\n[TEST 1] : Même Point")
    
    # Coordonnées de l'Arc de Triomphe
    lat_arc = 48.8738, 2.2950
    
    distance = func_calculer_distance(lat_arc[0], lat_arc[1], lat_arc[0], lat_arc[1])
    
    # Vérification
    if abs(distance - 0.0) < TOLERANCE:
        print(f"✅ Réussi : Distance calculée : {distance:.4f} km. (Attendu : ~0)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.4f} km. (Attendu : 0)")


    # ==========================================================
    ## 🎯 SCÉNARIO 2 : Petite Distance (Paris - Montmartre)
    # Distance de référence Google Maps : ~4.2 km
    # ==========================================================
    print("\n[TEST 2] : Petite Distance (Arc de Triomphe à Montmartre)")

    lat_montmartre = 48.8867, 2.3431
    reference_distance = 3.7  # km

    distance = func_calculer_distance(lat_arc[0], lat_arc[1], lat_montmartre[0], lat_montmartre[1])

    # Vérification
    if abs(distance - reference_distance) < 0.2: # Tolérance élargie à 200m
        print(f"✅ Réussi : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")


    # ==========================================================
    ## 🎯 SCÉNARIO 3 : Grande Distance (Paris - New York)
    # Distance de référence Google Maps : ~5837 km (selon la formule Haversine, cette valeur est très précise)
    # ==========================================================
    print("\n[TEST 3] : Grande Distance (Paris à New York)")
    
    lat_ny = 40.7128, -74.0060 # Lower Manhattan
    reference_distance = 5837.2 # km (Distance orthodromique)

    distance = func_calculer_distance(paris_lat, paris_lon, lat_ny[0], lat_ny[1])

    # Vérification (Tolérance de 1 km sur une grande distance)
    if abs(distance - reference_distance) < 1.0:
        print(f"✅ Réussi : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
        
    print("\n--- ✅ Fin des Tests ---")

# Définition des coordonnées de base
paris_lat, paris_lon = 48.8566, 2.3522 # Notre-Dame

# Exécution
func_test_calculer_distance()

--- 🧪 Début des Tests Manuels de Distance Haversine ---

[TEST 1] : Même Point
✅ Réussi : Distance calculée : 0.0000 km. (Attendu : ~0)

[TEST 2] : Petite Distance (Arc de Triomphe à Montmartre)
✅ Réussi : Distance calculée : 3.80 km. (Référence : 3.7 km)

[TEST 3] : Grande Distance (Paris à New York)
✅ Réussi : Distance calculée : 5837.24 km. (Référence : 5837.2 km)

--- ✅ Fin des Tests ---


In [52]:
def func_test_matching():
    nom_fichier = 'data.json'
    
    # 1. SPORTIF CHERCHEUR (ALICE)
    sportif_a = Sportif(
        prenom="Alice", sexe="Femme", age=28, localisalisation=[48.8, 2.3], distance_rencontre=20, 
        attentes="Performance", genre_recherche="Homme", min_age_recherchee=25, max_age_recherchee=35, 
        niveau_sports={"Running": "Avancé"}, nb_swipes_restants=10
    )

    # 2. DONNÉES DU FICHIER data.json
    donnees_json = [
        # SCÉNARIO 1 : MATCH PARFAIT (Bob)
        # OK: Attentes, OK: Âge, OK: Genre, OK: Distance
        {"prenom": "Bob", "sexe": "Homme", "age": 30, "localisalisation": [48.9, 2.4], "distance_rencontre": 30, "attentes": "Loisir", "genre_recherche": "Femme", "min_age_recherchee": 25, "max_age_recherchee": 35, "niveau_sports": {"Course": "Intermédiaire"}, "nb_swipes_restants": 10},
        
        # SCÉNARIO 2 : NON-MATCH (Âge)
        # Alice (28) est trop vieille pour la plage de Charlie (max 27)
        {"prenom": "Charlie", "sexe": "Homme", "age": 26, "localisalisation": [48.9, 2.4], "distance_rencontre": 30, "attentes": "Loisir", "genre_recherche": "Femme", "min_age_recherchee": 20, "max_age_recherchee": 27, "niveau_sports": {"Course": "Intermédiaire"}, "nb_swipes_restants": 10},
        
        # SCÉNARIO 3 : NON-MATCH (Distance)
        # Distance (100km) > Portée Alice (20km)
        {"prenom": "David", "sexe": "Homme", "age": 30, "localisalisation": [49.0, 2.5], "distance_rencontre": 30, "attentes": "Loisir", "genre_recherche": "Femme", "min_age_recherchee": 25, "max_age_recherchee": 35, "niveau_sports": {"Course": "Intermédiaire"}, "nb_swipes_restants": 10},
        
        # SCÉNARIO 4 : NON-MATCH (Genre mutuel)
        # Eric cherche "Homme" (pas "Femme")
        {"prenom": "Eric", "sexe": "Homme", "age": 30, "localisalisation": [48.9, 2.4], "distance_rencontre": 30, "attentes": "Loisir", "genre_recherche": "Homme", "min_age_recherchee": 25, "max_age_recherchee": 35, "niveau_sports": {"Course": "Intermédiaire"}, "nb_swipes_restants": 10},
        
        # SCÉNARIO 5 : NON-MATCH (Attentes)
        # Attentes identiques ("Performance")
        {"prenom": "Frank", "sexe": "Homme", "age": 30, "localisalisation": [48.9, 2.4], "distance_rencontre": 30, "attentes": "Performance", "genre_recherche": "Femme", "min_age_recherchee": 25, "max_age_recherchee": 35, "niveau_sports": {"Course": "Intermédiaire"}, "nb_swipes_restants": 10},
        
        # SCÉNARIO 6 : NON-MATCH (Swipes épuisés) - Testé via le filtre initial sur le sportif A
    ]

    # Préparation du fichier et exécution
    with open(nom_fichier, 'w', encoding='utf-8') as f:
        json.dump(donnees_json, f, indent=2)

    print("--- 🧪 Début des Tests de Matching (Version Finale) ---")
    
    # 1. SCÉNARIO PRINCIPAL
    resultats = func_algo_matching(sportif_a)
    noms_trouves = [s.prenom for s in resultats]
    noms_attendus = ["Bob"]
    
    print("\n--- Analyse des Résultats ---")
    
    if set(noms_trouves) == set(noms_attendus) and len(noms_trouves) == len(noms_attendus):
        print(f"✅ Réussi (Principal) : Le match trouvé est correct. Matchs: {noms_trouves}")
    else:
        print(f"❌ Échoué (Principal) : Matchs trouvés: {noms_trouves}. Matchs attendus: {noms_attendus}")
        
    # 2. TEST DU FILTRE DES SWIPES
    sportif_echec_swipes = Sportif(prenom="Gaspard", localisalisation=[0, 0], distance_rencontre=10, attentes="Loisir", genre_recherche="Femme", min_age_recherchee=20, max_age_recherchee=40, sexe="Homme", age=30, nb_swipes_restants=0)
    resultats_swipes = func_algo_matching(sportif_echec_swipes)
    
    if len(resultats_swipes) == 0:
        print("✅ Réussi (Swipes) : Le matching a été stoppé lorsque nb_swipes_restantes est à 0.")
    else:
        print("❌ Échoué (Swipes) : Le matching s'est poursuivi malgré l'épuisement des swipes.")
    
    # Nettoyage
    os.remove(nom_fichier)

# Exécution
func_test_matching()

TypeError: Sportif.__init__() got an unexpected keyword argument 'nb_swipes_restants'